# Complete Stock Analysis Example
## Learning Fundamental and Technical Indicators

This notebook demonstrates how to analyze a stock using both fundamental indicators (from financial statements) and technical indicators (from price/volume data).

**What you'll learn:**
1. How to fetch real stock data using yfinance
2. Calculating valuation ratios (P/E, P/B, EV/EBITDA)
3. Calculating profitability ratios (ROE, ROA, margins)
4. Technical indicators (RSI, MACD, Moving Averages)
5. Composite scores (Piotroski F-Score)
6. Visualizing indicators on charts

**Disclaimer:** This is for educational purposes only. Not financial advice.

## Part 1: Setup and Imports

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set display options for better readability
pd.set_option('display.float_format', '{:.2f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully!")

## Part 2: Fetching Real Stock Data

We'll use Apple (AAPL) as our example. The `yfinance` library gives us:
- `info`: Company fundamentals (P/E, market cap, etc.)
- `history`: Historical price and volume data
- `financials`: Income statement data
- `balance_sheet`: Assets and liabilities
- `cashflow`: Cash flow statement

In [ ]:
# Fetch Apple stock data
ticker_symbol = "AAPL"
ticker = yf.Ticker(ticker_symbol)

# Get different types of data
info = ticker.info
history = ticker.history(period="1y")  # 1 year of price data

print(f"Analyzing: {info.get('longName', ticker_symbol)}")
print(f"Sector: {info.get('sector', 'N/A')}")
print(f"Industry: {info.get('industry', 'N/A')}")
print(f"Current Price: ${info.get('currentPrice', 'N/A')}")
print(f"Market Cap: ${info.get('marketCap', 0) / 1e9:.2f} Billion")

## Part 3: Valuation Ratios (Fundamental Analysis)

Valuation ratios help answer: **"Is this stock expensive or cheap relative to what you're getting?"**

### P/E Ratio (Price-to-Earnings)
- **Formula:** Stock Price ÷ Earnings Per Share
- **What it means:** How many dollars investors pay for $1 of earnings
- **Analogy:** Like asking "how many years of profit would it take to pay off the stock price?"
- **Interpretation:** Lower = potentially cheaper, Higher = growth expectations or overvalued

In [ ]:
# Calculate P/E Ratio manually to understand it
stock_price = info.get('currentPrice', 0)
eps = info.get('trailingEps', 0)  # Earnings Per Share (last 12 months)

# Manual calculation
pe_ratio_manual = stock_price / eps if eps > 0 else None

# Yahoo Finance also provides it directly
pe_ratio_yahoo = info.get('trailingPE', 0)

print("=== P/E Ratio Analysis ===")
print(f"Stock Price: ${stock_price:.2f}")
print(f"Earnings Per Share (TTM): ${eps:.2f}")
print(f"P/E Ratio (calculated): {pe_ratio_manual:.2f}")
print(f"P/E Ratio (from Yahoo): {pe_ratio_yahoo:.2f}")
print()
print("Interpretation:")
print(f"  Investors pay ${pe_ratio_manual:.2f} for every $1 of Apple's earnings")
print(f"  Tech sector average: ~25-35")
if pe_ratio_manual:
    if pe_ratio_manual < 15:
        print("  This P/E is LOW - could be undervalued or declining business")
    elif pe_ratio_manual < 25:
        print("  This P/E is MODERATE - fairly valued for a stable company")
    elif pe_ratio_manual < 40:
        print("  This P/E is HIGH - growth expectations priced in")
    else:
        print("  This P/E is VERY HIGH - extremely optimistic expectations")

### Other Key Valuation Ratios

| Ratio | Formula | Best For |
|-------|---------|----------|
| P/B | Price ÷ Book Value | Banks, asset-heavy companies |
| P/S | Price ÷ Sales | Unprofitable growth companies |
| EV/EBITDA | Enterprise Value ÷ EBITDA | Comparing across capital structures |

In [ ]:
def calculate_valuation_ratios(ticker_info):
    """
    Calculate key valuation ratios from Yahoo Finance data.
    Returns a dictionary with ratio names and values.
    """
    ratios = {
        'P/E (Trailing)': ticker_info.get('trailingPE'),
        'P/E (Forward)': ticker_info.get('forwardPE'),
        'P/B (Price-to-Book)': ticker_info.get('priceToBook'),
        'P/S (Price-to-Sales)': ticker_info.get('priceToSalesTrailing12Months'),
        'EV/EBITDA': ticker_info.get('enterpriseToEbitda'),
        'EV/Revenue': ticker_info.get('enterpriseToRevenue'),
        'PEG Ratio': ticker_info.get('pegRatio'),
        'Dividend Yield (%)': (ticker_info.get('dividendYield') or 0) * 100
    }
    return pd.Series(ratios)

valuation_ratios = calculate_valuation_ratios(info)
print("=== Valuation Ratios Dashboard ===")
print(valuation_ratios.to_string())
print()
print("Quick Interpretation Guide:")
print("  P/E: Lower is cheaper (compare within same industry)")
print("  P/B: Below 1 means trading below book value (rare)")
print("  EV/EBITDA: 8-12 is typical; lower = potentially undervalued")
print("  PEG < 1: Growth at a reasonable price")

## Part 4: Profitability Ratios

Profitability ratios answer: **"How good is this company at making money?"**

### ROE (Return on Equity)
- **Formula:** Net Income ÷ Shareholders' Equity
- **What it means:** How much profit the company generates with shareholder money
- **Analogy:** Like asking "if I invested $100, how much profit would be generated?"
- **Good benchmark:** >15% is generally strong

In [ ]:
def calculate_profitability_ratios(ticker_info):
    """
    Calculate key profitability ratios.
    These measure how efficiently a company generates profit.
    """
    ratios = {
        'Gross Margin (%)': (ticker_info.get('grossMargins') or 0) * 100,
        'Operating Margin (%)': (ticker_info.get('operatingMargins') or 0) * 100,
        'Net Profit Margin (%)': (ticker_info.get('profitMargins') or 0) * 100,
        'ROE (%)': (ticker_info.get('returnOnEquity') or 0) * 100,
        'ROA (%)': (ticker_info.get('returnOnAssets') or 0) * 100,
    }
    return pd.Series(ratios)

profitability = calculate_profitability_ratios(info)
print("=== Profitability Ratios ===")
print(profitability.to_string())
print()
print("Interpretation:")
print("  Gross Margin: % of revenue after cost of goods")
print("  Operating Margin: % after operating expenses")
print("  Net Margin: % that becomes actual profit")
print("  ROE > 15%: Strong return on shareholder investment")
print("  ROA > 5%: Good asset utilization")

## Part 5: Technical Indicators (Price Analysis)

Technical indicators use **price and volume** to identify trends and momentum.

### Moving Averages (SMA & EMA)
- **SMA (Simple Moving Average):** Average price over N days
- **EMA (Exponential Moving Average):** Weighted average (recent prices matter more)
- **Uses:** Trend direction, support/resistance levels

In [ ]:
# Calculate Moving Averages
history['SMA_20'] = history['Close'].rolling(window=20).mean()  # 20-day SMA
history['SMA_50'] = history['Close'].rolling(window=50).mean()  # 50-day SMA
history['EMA_12'] = history['Close'].ewm(span=12, adjust=False).mean()  # 12-day EMA
history['EMA_26'] = history['Close'].ewm(span=26, adjust=False).mean()  # 26-day EMA

# Show recent values
print("=== Moving Averages (Last 5 Days) ===")
print(history[['Close', 'SMA_20', 'SMA_50', 'EMA_12', 'EMA_26']].tail())
print()

# Trend interpretation
current_price = history['Close'].iloc[-1]
sma_20 = history['SMA_20'].iloc[-1]
sma_50 = history['SMA_50'].iloc[-1]

print("Trend Signals:")
if current_price > sma_20 > sma_50:
    print("  BULLISH: Price above both moving averages")
elif current_price < sma_20 < sma_50:
    print("  BEARISH: Price below both moving averages")
else:
    print("  MIXED: No clear trend signal")

### RSI (Relative Strength Index)

- **Formula:** 100 - (100 / (1 + RS)), where RS = Avg Gain / Avg Loss
- **Range:** 0 to 100
- **Interpretation:**
  - RSI > 70: **Overbought** (price may be too high, could reverse down)
  - RSI < 30: **Oversold** (price may be too low, could reverse up)
  - RSI 40-60: **Neutral** zone

In [ ]:
def calculate_rsi(prices, period=14):
    """
    Calculate Relative Strength Index (RSI).
    
    RSI measures momentum - whether price is rising or falling too fast.
    
    Parameters:
    - prices: Series of closing prices
    - period: Lookback period (default 14 days)
    
    Returns:
    - Series of RSI values (0-100)
    """
    # Calculate price changes
    delta = prices.diff()
    
    # Separate gains (positive changes) and losses (negative changes)
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)
    
    # Calculate average gains and losses
    avg_gain = gains.rolling(window=period).mean()
    avg_loss = losses.rolling(window=period).mean()
    
    # Calculate Relative Strength (RS)
    rs = avg_gain / avg_loss
    
    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))
    
    return rsi

# Calculate RSI
history['RSI'] = calculate_rsi(history['Close'])

# Current RSI
current_rsi = history['RSI'].iloc[-1]
print(f"=== RSI Analysis ===")
print(f"Current RSI: {current_rsi:.2f}")
print()

if current_rsi > 70:
    print("Status: OVERBOUGHT")
    print("  The stock has been rising strongly.")
    print("  It MAY be due for a pullback (not guaranteed).")
elif current_rsi < 30:
    print("Status: OVERSOLD")
    print("  The stock has been falling strongly.")
    print("  It MAY be due for a bounce (not guaranteed).")
else:
    print("Status: NEUTRAL")
    print("  No extreme momentum signal.")

### MACD (Moving Average Convergence Divergence)

- **Components:**
  - MACD Line: 12-day EMA - 26-day EMA
  - Signal Line: 9-day EMA of MACD Line
  - Histogram: MACD Line - Signal Line
- **Signals:**
  - MACD crosses ABOVE signal: **Bullish**
  - MACD crosses BELOW signal: **Bearish**

In [ ]:
def calculate_macd(prices, fast=12, slow=26, signal=9):
    """
    Calculate MACD (Moving Average Convergence Divergence).
    
    MACD shows the relationship between two moving averages.
    It helps identify trend direction and momentum changes.
    """
    # Calculate EMAs
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    
    # MACD Line = Fast EMA - Slow EMA
    macd_line = ema_fast - ema_slow
    
    # Signal Line = 9-day EMA of MACD Line
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    
    # Histogram = MACD Line - Signal Line
    histogram = macd_line - signal_line
    
    return macd_line, signal_line, histogram

# Calculate MACD
history['MACD'], history['MACD_Signal'], history['MACD_Hist'] = calculate_macd(history['Close'])

# Current values
current_macd = history['MACD'].iloc[-1]
current_signal = history['MACD_Signal'].iloc[-1]
current_hist = history['MACD_Hist'].iloc[-1]

print("=== MACD Analysis ===")
print(f"MACD Line: {current_macd:.4f}")
print(f"Signal Line: {current_signal:.4f}")
print(f"Histogram: {current_hist:.4f}")
print()

if current_macd > current_signal:
    print("Signal: BULLISH (MACD above Signal line)")
else:
    print("Signal: BEARISH (MACD below Signal line)")

if current_hist > 0:
    print("Momentum: Positive and potentially strengthening")
else:
    print("Momentum: Negative and potentially weakening")

### Bollinger Bands (Volatility)

- **Components:**
  - Middle Band: 20-day SMA
  - Upper Band: Middle + (2 × Standard Deviation)
  - Lower Band: Middle - (2 × Standard Deviation)
- **Uses:** Measure volatility, identify potential breakouts

In [ ]:
def calculate_bollinger_bands(prices, period=20, std_dev=2):
    """
    Calculate Bollinger Bands.
    
    Bollinger Bands show volatility - wider bands = more volatile.
    Price near upper band = potentially overbought
    Price near lower band = potentially oversold
    """
    # Middle Band = SMA
    middle = prices.rolling(window=period).mean()
    
    # Standard Deviation
    std = prices.rolling(window=period).std()
    
    # Upper and Lower Bands
    upper = middle + (std * std_dev)
    lower = middle - (std * std_dev)
    
    return upper, middle, lower

# Calculate Bollinger Bands
history['BB_Upper'], history['BB_Middle'], history['BB_Lower'] = calculate_bollinger_bands(history['Close'])

# Current position within bands
current_price = history['Close'].iloc[-1]
bb_upper = history['BB_Upper'].iloc[-1]
bb_lower = history['BB_Lower'].iloc[-1]
bb_middle = history['BB_Middle'].iloc[-1]

# Calculate % position within bands (0 = at lower, 100 = at upper)
bb_position = ((current_price - bb_lower) / (bb_upper - bb_lower)) * 100

print("=== Bollinger Bands Analysis ===")
print(f"Upper Band: ${bb_upper:.2f}")
print(f"Middle Band (SMA): ${bb_middle:.2f}")
print(f"Lower Band: ${bb_lower:.2f}")
print(f"Current Price: ${current_price:.2f}")
print(f"Position within bands: {bb_position:.1f}%")
print()

if bb_position > 80:
    print("Price is NEAR UPPER BAND - potentially overbought")
elif bb_position < 20:
    print("Price is NEAR LOWER BAND - potentially oversold")
else:
    print("Price is in the MIDDLE of the bands - neutral")

## Part 6: Composite Scores

Composite scores combine multiple indicators into a single metric.

### Piotroski F-Score (0-9)

The F-Score tests 9 criteria across profitability, leverage, and efficiency.
- **Score 0-2:** Weak fundamentals
- **Score 3-5:** Average
- **Score 6-9:** Strong fundamentals

In [ ]:
def calculate_piotroski_f_score(ticker_info):
    """
    Calculate Piotroski F-Score (simplified version).
    
    The F-Score tests 9 binary conditions:
    - Profitability (4 points)
    - Leverage/Liquidity (3 points)
    - Operating Efficiency (2 points)
    
    Returns score (0-9) and breakdown.
    """
    score = 0
    breakdown = {}
    
    # PROFITABILITY (4 criteria)
    
    # 1. ROA > 0 (positive net income)
    roa = ticker_info.get('returnOnAssets', 0) or 0
    if roa > 0:
        score += 1
        breakdown['ROA > 0'] = 'PASS'
    else:
        breakdown['ROA > 0'] = 'FAIL'
    
    # 2. Operating Cash Flow > 0
    ocf = ticker_info.get('operatingCashflow', 0) or 0
    if ocf > 0:
        score += 1
        breakdown['Operating CF > 0'] = 'PASS'
    else:
        breakdown['Operating CF > 0'] = 'FAIL'
    
    # 3. Cash Flow > Net Income (quality of earnings)
    net_income = ticker_info.get('netIncomeToCommon', 0) or 0
    if ocf > net_income:
        score += 1
        breakdown['Cash Flow > Net Income'] = 'PASS'
    else:
        breakdown['Cash Flow > Net Income'] = 'FAIL'
    
    # LEVERAGE (simplified - checking debt levels)
    
    # 4. Current Ratio > 1 (can pay short-term debts)
    current_ratio = ticker_info.get('currentRatio', 0) or 0
    if current_ratio > 1:
        score += 1
        breakdown['Current Ratio > 1'] = 'PASS'
    else:
        breakdown['Current Ratio > 1'] = 'FAIL'
    
    # 5. Positive profit margins
    profit_margin = ticker_info.get('profitMargins', 0) or 0
    if profit_margin > 0:
        score += 1
        breakdown['Profit Margin > 0'] = 'PASS'
    else:
        breakdown['Profit Margin > 0'] = 'FAIL'
    
    # EFFICIENCY
    
    # 6. Gross Margin positive
    gross_margin = ticker_info.get('grossMargins', 0) or 0
    if gross_margin > 0:
        score += 1
        breakdown['Gross Margin > 0'] = 'PASS'
    else:
        breakdown['Gross Margin > 0'] = 'FAIL'
    
    return score, breakdown

# Calculate F-Score
f_score, breakdown = calculate_piotroski_f_score(info)

print("=== Piotroski F-Score Analysis ===")
print(f"F-Score: {f_score}/6 (simplified version)")
print()
print("Breakdown:")
for criterion, result in breakdown.items():
    status = '+1' if result == 'PASS' else ' 0'
    print(f"  [{status}] {criterion}: {result}")
print()

if f_score >= 5:
    print("Assessment: STRONG fundamentals")
elif f_score >= 3:
    print("Assessment: AVERAGE fundamentals")
else:
    print("Assessment: WEAK fundamentals (investigate further)")

## Part 7: Complete Visualization Dashboard

Let's create a comprehensive chart showing price with all our indicators.

In [ ]:
# Create a 4-panel dashboard
fig, axes = plt.subplots(4, 1, figsize=(14, 16), sharex=True)
fig.suptitle(f'{ticker_symbol} Technical Analysis Dashboard', fontsize=16, fontweight='bold')

# Panel 1: Price with Moving Averages and Bollinger Bands
ax1 = axes[0]
ax1.plot(history.index, history['Close'], label='Price', color='black', linewidth=1.5)
ax1.plot(history.index, history['SMA_20'], label='SMA 20', color='blue', alpha=0.7)
ax1.plot(history.index, history['SMA_50'], label='SMA 50', color='orange', alpha=0.7)
ax1.fill_between(history.index, history['BB_Upper'], history['BB_Lower'], 
                  alpha=0.2, color='gray', label='Bollinger Bands')
ax1.set_ylabel('Price ($)')
ax1.legend(loc='upper left')
ax1.set_title('Price with Moving Averages & Bollinger Bands')

# Panel 2: Volume
ax2 = axes[1]
colors = ['green' if history['Close'].iloc[i] >= history['Open'].iloc[i] else 'red' 
          for i in range(len(history))]
ax2.bar(history.index, history['Volume'], color=colors, alpha=0.7)
ax2.set_ylabel('Volume')
ax2.set_title('Trading Volume')

# Panel 3: RSI
ax3 = axes[2]
ax3.plot(history.index, history['RSI'], color='purple', linewidth=1.5)
ax3.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
ax3.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
ax3.axhline(y=50, color='gray', linestyle=':', alpha=0.5)
ax3.fill_between(history.index, 70, 100, alpha=0.1, color='red')
ax3.fill_between(history.index, 0, 30, alpha=0.1, color='green')
ax3.set_ylabel('RSI')
ax3.set_ylim(0, 100)
ax3.legend(loc='upper left')
ax3.set_title('Relative Strength Index (RSI)')

# Panel 4: MACD
ax4 = axes[3]
ax4.plot(history.index, history['MACD'], label='MACD', color='blue', linewidth=1.5)
ax4.plot(history.index, history['MACD_Signal'], label='Signal', color='orange', linewidth=1.5)
colors_macd = ['green' if val >= 0 else 'red' for val in history['MACD_Hist']]
ax4.bar(history.index, history['MACD_Hist'], color=colors_macd, alpha=0.5, label='Histogram')
ax4.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax4.set_ylabel('MACD')
ax4.legend(loc='upper left')
ax4.set_title('MACD (Moving Average Convergence Divergence)')

plt.tight_layout()
plt.show()

print("\n" + "="*60)
print("DASHBOARD READING GUIDE")
print("="*60)
print("Panel 1 (Price):")
print("  - Price above moving averages = uptrend")
print("  - Price touching Bollinger Bands = potential reversal")
print("\nPanel 2 (Volume):")
print("  - High volume confirms price moves")
print("  - Green = up day, Red = down day")
print("\nPanel 3 (RSI):")
print("  - Above 70 (red zone) = overbought")
print("  - Below 30 (green zone) = oversold")
print("\nPanel 4 (MACD):")
print("  - Blue above Orange = bullish")
print("  - Green histogram = strengthening, Red = weakening")

## Part 8: Analysis Summary

Let's bring all our analysis together into a summary report.

In [ ]:
print("="*70)
print(f"STOCK ANALYSIS SUMMARY: {info.get('longName', ticker_symbol)}")
print("="*70)

print("\n--- FUNDAMENTAL ANALYSIS ---")
print(f"P/E Ratio: {info.get('trailingPE', 'N/A'):.2f}")
print(f"P/B Ratio: {info.get('priceToBook', 'N/A'):.2f}")
print(f"ROE: {(info.get('returnOnEquity', 0) or 0) * 100:.1f}%")
print(f"Profit Margin: {(info.get('profitMargins', 0) or 0) * 100:.1f}%")
print(f"Piotroski F-Score: {f_score}/6")

print("\n--- TECHNICAL ANALYSIS ---")
print(f"Current Price: ${current_price:.2f}")
print(f"RSI (14-day): {current_rsi:.2f}")
print(f"MACD Signal: {'Bullish' if current_macd > current_signal else 'Bearish'}")
print(f"Bollinger Band Position: {bb_position:.1f}%")

print("\n--- TREND ANALYSIS ---")
if current_price > sma_20 > sma_50:
    print("Trend: UPTREND (price above 20 & 50 day averages)")
elif current_price < sma_20 < sma_50:
    print("Trend: DOWNTREND (price below 20 & 50 day averages)")
else:
    print("Trend: SIDEWAYS/MIXED")

print("\n" + "="*70)
print("DISCLAIMER: This is for educational purposes only.")
print("Always do your own research and consult a financial advisor.")
print("="*70)

## Your Turn: Experiment!

Try changing the `ticker_symbol` at the top of this notebook to analyze different stocks:

- **Tech:** MSFT, GOOGL, NVDA, META
- **Finance:** JPM, BAC, GS
- **Consumer:** AMZN, WMT, KO
- **Healthcare:** JNJ, PFE, UNH

**Questions to explore:**
1. How does P/E vary across different sectors?
2. Do tech stocks have higher or lower profit margins than banks?
3. Can you find a stock with RSI below 30 (oversold)?

In [ ]:
# EXPERIMENT: Change this ticker and run all cells above again!
# Try: "MSFT", "GOOGL", "JPM", "KO", etc.

experiment_ticker = "MSFT"  # Change this!

# Quick analysis
exp = yf.Ticker(experiment_ticker)
exp_info = exp.info

print(f"Quick Look: {exp_info.get('longName', experiment_ticker)}")
print(f"Sector: {exp_info.get('sector', 'N/A')}")
print(f"P/E: {exp_info.get('trailingPE', 'N/A')}")
print(f"ROE: {(exp_info.get('returnOnEquity', 0) or 0) * 100:.1f}%")